In [1]:
import os

os.chdir('/mnt/jw01-aruk-home01/projects/ra_challenge/RA_challenge/michael_dev/RA2_alpine_lads/ra_joint_predictions')

In [2]:
from utils.config import Config

config = Config()

In [3]:
import numpy as np
import tensorflow as tf

from model.utils.metrics import argmax_rsme, softmax_rsme_metric, class_softmax_rsme_metric

dependencies = {
    'softmax_rsme': softmax_rsme_metric(np.arange(5)),
    'argmax_rsme': argmax_rsme,
    'class_softmax_rmse_0': class_softmax_rsme_metric(np.arange(5), 0)
}

narrowing_model = tf.keras.models.load_model('./pretrained_models/joint_erosion_model_v1.h5', custom_objects=dependencies)

In [4]:
import numpy as np
import dataset.joint_dataset as joint_dataset
from dataset.test_dataset import joint_test_dataset

dataset = joint_test_dataset(config, config.train_fixed_location)
feet_test_dataset = dataset.get_feet_joint_test_dataset()

preds = {}

for file_info, img in feet_test_dataset:
    file_info = file_info.numpy()
    
    img_info = file_info[0].decode('utf-8').split('.')[0]
    key = file_info[3].decode('utf-8')
    
    patient_info = img_info.split('-')
    patient_id = patient_info[0]
    part = patient_info[1]
    
    # If patient_id is not found, init with empty struct
    if not patient_id in preds.keys():
        preds[patient_id] = {
            'Patient_ID': patient_id
        }
        
    outcome_key = joint_dataset.foot_outcome_mapping[key][0][0].format(part = part)
        
    y_pred = narrowing_model.predict(tf.expand_dims(img, 0))
    y_pred = np.sum(y_pred * np.arange(5))
    
    preds[patient_id][outcome_key] = y_pred

In [5]:
import pandas as pd

x = pd.DataFrame(preds.values(), index = np.arange(len(preds.values())))

In [6]:
narrowing_mask = ['_J_' in column_name for column_name in x.columns]
narrowing_sum = np.sum(x.iloc[:, narrowing_mask].to_numpy(), axis = 1)

x['Overall_narrowing'] = narrowing_sum

In [7]:
true = pd.read_csv(config.train_location + '/training.csv')

In [8]:
mask = [patient_id in x['Patient_ID'].to_numpy() for patient_id in true['Patient_ID']]

true_part = true.iloc[mask]

In [14]:
cols = ['LF_mtp_J__1', 'LF_mtp_J__2', 'LF_mtp_J__3', 'LF_mtp_J__4','LF_mtp_J__5', 'LF_mtp_J__ip','RF_mtp_J__1','RF_mtp_J__2','RF_mtp_J__3', 'RF_mtp_J__4','RF_mtp_J__5','RF_mtp_J__ip']

pred = x[cols].to_numpy()
truth = true_part[cols].to_numpy()

In [15]:
rsme = np.sqrt(np.mean(np.square(pred - truth), axis = 0))

In [16]:
print(rsme)

[1.09348031 1.00434937 1.17243929 1.14839225 1.16737918 0.79881684
 1.20975802 1.24169407 1.33511884 1.3210651  1.47863203 0.99052181]


In [19]:
ov_narrow = np.sum(pred, axis = 1)
ov_true_narrow = np.sum(truth, axis = 1)

In [20]:
ov_rsme = np.sqrt(np.mean(np.square(ov_narrow - ov_true_narrow), axis = 0))

print(ov_rsme)

10.101996592949074
